In [2]:
import org.apache.spark.sql.SparkSession
import org.apache.spark.sql.types._
import org.apache.spark.sql.functions._
import org.apache.spark.sql._

val spark = SparkSession.builder()
.appName("Joins")
.getOrCreate()

import org.apache.spark.sql.SparkSession
import org.apache.spark.sql.types._
import org.apache.spark.sql.functions._
import org.apache.spark.sql._
spark: org.apache.spark.sql.SparkSession = org.apache.spark.sql.SparkSession@739c8c3c


In [3]:
val band = spark.read.option("inferSchema", "true").json("./dataset/bands.json")
val player = spark.read.option("inferSchema", "true").json("./dataset/guitarPlayers.json")
val guitar = spark.read.option("inferSchema", "true").json("./dataset/guitars.json")

band: org.apache.spark.sql.DataFrame = [hometown: string, id: bigint ... 2 more fields]
player: org.apache.spark.sql.DataFrame = [band: bigint, guitars: array<bigint> ... 2 more fields]
guitar: org.apache.spark.sql.DataFrame = [guitarType: string, id: bigint ... 2 more fields]


In [4]:
println("band")
band.show(5)

println("player")
player.show(5)

println("guitar")
guitar.show(5)

band
+-----------+---+------------+----+
|   hometown| id|        name|year|
+-----------+---+------------+----+
|     Sydney|  1|       AC/DC|1973|
|     London|  0|Led Zeppelin|1968|
|Los Angeles|  3|   Metallica|1981|
|  Liverpool|  4| The Beatles|1960|
+-----------+---+------------+----+

player
+----+-------+---+------------+
|band|guitars| id|        name|
+----+-------+---+------------+
|   0|    [0]|  0|  Jimmy Page|
|   1|    [1]|  1| Angus Young|
|   2| [1, 5]|  2|Eric Clapton|
|   3|    [3]|  3|Kirk Hammett|
+----+-------+---+------------+

guitar
+--------------------+---+------+------------+
|          guitarType| id|  make|       model|
+--------------------+---+------+------------+
|Electric double-n...|  0|Gibson|    EDS-1275|
|            Electric|  5|Fender|Stratocaster|
|            Electric|  1|Gibson|          SG|
|            Acoustic|  2|Taylor|         914|
|            Electric|  3|   ESP|        M-II|
+--------------------+---+------+------------+



In [24]:
// 0. JOIN CONDITION

val joinCondition = Seq("id")

joinCondition: Seq[String] = List(id)


In [30]:
// 1. INNER
// val bandPlayer = band.join(player, band.col("id") === player.col("id"))
val bandPlayer = band.join(player, joinCondition)
bandPlayer.show()

+---+-----------+------------+----+----+-------+------------+
| id|   hometown|        name|year|band|guitars|        name|
+---+-----------+------------+----+----+-------+------------+
|  1|     Sydney|       AC/DC|1973|   1|    [1]| Angus Young|
|  0|     London|Led Zeppelin|1968|   0|    [0]|  Jimmy Page|
|  3|Los Angeles|   Metallica|1981|   3|    [3]|Kirk Hammett|
+---+-----------+------------+----+----+-------+------------+



bandPlayer: org.apache.spark.sql.DataFrame = [id: bigint, hometown: string ... 5 more fields]


In [31]:
// 2. LEFT OUTER
band.join(player, joinCondition, "left_outer").show()

+---+-----------+------------+----+----+-------+------------+
| id|   hometown|        name|year|band|guitars|        name|
+---+-----------+------------+----+----+-------+------------+
|  1|     Sydney|       AC/DC|1973|   1|    [1]| Angus Young|
|  0|     London|Led Zeppelin|1968|   0|    [0]|  Jimmy Page|
|  3|Los Angeles|   Metallica|1981|   3|    [3]|Kirk Hammett|
|  4|  Liverpool| The Beatles|1960|NULL|   NULL|        NULL|
+---+-----------+------------+----+----+-------+------------+



In [32]:
// 3. RIGHT OUTER
band.join(player, joinCondition, "right_outer").show()

+---+-----------+------------+----+----+-------+------------+
| id|   hometown|        name|year|band|guitars|        name|
+---+-----------+------------+----+----+-------+------------+
|  0|     London|Led Zeppelin|1968|   0|    [0]|  Jimmy Page|
|  1|     Sydney|       AC/DC|1973|   1|    [1]| Angus Young|
|  2|       NULL|        NULL|NULL|   2| [1, 5]|Eric Clapton|
|  3|Los Angeles|   Metallica|1981|   3|    [3]|Kirk Hammett|
+---+-----------+------------+----+----+-------+------------+



In [33]:
// 4. OUTER
band.join(player, joinCondition, "outer").show()

+---+-----------+------------+----+----+-------+------------+
| id|   hometown|        name|year|band|guitars|        name|
+---+-----------+------------+----+----+-------+------------+
|  0|     London|Led Zeppelin|1968|   0|    [0]|  Jimmy Page|
|  1|     Sydney|       AC/DC|1973|   1|    [1]| Angus Young|
|  2|       NULL|        NULL|NULL|   2| [1, 5]|Eric Clapton|
|  3|Los Angeles|   Metallica|1981|   3|    [3]|Kirk Hammett|
|  4|  Liverpool| The Beatles|1960|NULL|   NULL|        NULL|
+---+-----------+------------+----+----+-------+------------+



In [34]:
// 5. LEFT SEMI
band.join(player, joinCondition, "left_semi").show()

+---+-----------+------------+----+
| id|   hometown|        name|year|
+---+-----------+------------+----+
|  1|     Sydney|       AC/DC|1973|
|  0|     London|Led Zeppelin|1968|
|  3|Los Angeles|   Metallica|1981|
+---+-----------+------------+----+



In [35]:
// 6. LEFT ANITA
band.join(player, joinCondition, "left_anti").show()

+---+---------+-----------+----+
| id| hometown|       name|year|
+---+---------+-----------+----+
|  4|Liverpool|The Beatles|1960|
+---+---------+-----------+----+



In [ ]:
// inner joins
val joinCondition = player.col("band") === band.col("id")
val guitaristsBandsDF = player.join(band, joinCondition, "inner")

// outer joins
// left outer = everything in the inner join + all the rows in the LEFT table, with nulls in where the data is missing
player.join(band, joinCondition, "left_outer")

// right outer = everything in the inner join + all the rows in the RIGHT table, with nulls in where the data is missing
player.join(band, joinCondition, "right_outer")

// outer join = everything in the inner join + all the rows in BOTH tables, with nulls in where the data is missing
player.join(band, joinCondition, "outer")

// semi-joins = everything in the left DF for which there is a row in the right DF satisfying the condition
player.join(band, joinCondition, "left_semi")

// anti-joins = everything in the left DF for which there is NO row in the right DF satisfying the condition
player.join(band, joinCondition, "left_anti")

In [8]:
player.join(guitar.withColumnRenamed("id", "guitarId"), expr("array_contains(guitars, guitarId)")).show()

+----+-------+---+------------+--------------------+--------+------+------------+
|band|guitars| id|        name|          guitarType|guitarId|  make|       model|
+----+-------+---+------------+--------------------+--------+------+------------+
|   0|    [0]|  0|  Jimmy Page|Electric double-n...|       0|Gibson|    EDS-1275|
|   2| [1, 5]|  2|Eric Clapton|            Electric|       5|Fender|Stratocaster|
|   1|    [1]|  1| Angus Young|            Electric|       1|Gibson|          SG|
|   2| [1, 5]|  2|Eric Clapton|            Electric|       1|Gibson|          SG|
|   3|    [3]|  3|Kirk Hammett|            Electric|       3|   ESP|        M-II|
+----+-------+---+------------+--------------------+--------+------+------------+

